In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

#### Utility to generate random BST

In [4]:
def generateBST(N: int, min_: int, max_: int) -> BinaryTreeNode|None:
    def insert(curr: BinaryTreeNode|None, n: int) -> BinaryTreeNode:
        if not curr:
            curr = BinaryTreeNode(n)
        elif curr.val < n:
            curr.right = insert(curr.right, n)
        else:
            curr.left = insert(curr.left, n)

        return curr

    assert N <= max_ - min_, "Number of available samples must be >= N"
    root: BinaryTreeNode|None = None
    for n in np.random.choice(np.arange(min_, max_), size=N, replace=False):
        root = insert(root, n)

    return root

GFG Jobathon: 32

Q1: Lead Engineer

In [5]:
def leadOptions(self, N : int, E : List[int]) -> int:
    maxE: tuple[int, int] = -1, 0
    for exp in E:
        if exp == maxE[0]:
            maxE = maxE[0], maxE[1] + 1
        if exp > maxE[0]:
            maxE = exp, 0

    return maxE[1]

Codeforces: 20/C (Dijkstra)

In [6]:
def dijkstra(src: int, adjl: dict[int, list[tuple[int, int]]]) -> dict[int, tuple[float, int]]:
    heap: list[tuple[int, int, int]] = [(1, src, 1)]
    shortest: dict[int, tuple[float, int]] = dict()
    while heap:
        curr_dist, curr, prev = heapq.heappop(heap)
        if curr not in shortest or curr_dist < shortest[curr][0]:
            shortest[curr] = (curr_dist, prev)
            for next_, next_dist in adjl.get(curr, []):
                heapq.heappush(heap, (curr_dist + next_dist, next_, curr))

    return shortest

def twentyC(V: int, E: int, edges: list[tuple[int, int, int]]) -> str:
    adjl: dict[int, list[tuple[int, int]]] = dict()
    for n1, n2, d in edges:
        n1_neighbours, n2_neighbours = adjl.get(n1, []), adjl.get(n2, [])
        n1_neighbours.append((n2, d))
        n2_neighbours.append((n1, d))
        adjl[n1], adjl[n2] = n1_neighbours, n2_neighbours

    paths = dijkstra(1, adjl)
    if V in paths:
        result: list[int] = [V]
        curr = V
        while curr != 1:
            dist, prev = paths[curr]
            result.append(prev)
            curr = prev

        result.reverse()
        return ' '.join(map(str, result))
    else:
        return '-1'

# Testing the solution
assert twentyC(5, 5, [(1, 2, 2), (2, 5, 5), (2, 3, 4), (1, 4, 1), (4, 3, 3), (3, 5, 1)]) == '1 4 3 5'

LC Medium: Course Schedule 2

In [7]:
# https://leetcode.com/problems/course-schedule-ii/submissions/1239496066
def findOrder(numCourses: int, prerequisites: list[list[int]]) -> list[int]:
    adjl: dict[int, list[int]] = dict()
    for c1, c2 in prerequisites:
        # To take up C1, C2 must be done already
        c1_neighbours: list[int] = adjl.get(c1, [])
        c1_neighbours.append(c2)
        adjl[c1] = c1_neighbours

    paths: dict[int, list[int]] = dict()
    def getOrdering(curr: int, visited: set[int] = set()) -> list[int]:
        if curr in paths:
            return paths[curr]
        elif curr in visited:
            return []
        else:
            result: list[int] = [curr]
            visited.add(curr)
            for next_ in adjl.get(curr, []):
                deps: list[int] = getOrdering(next_, visited)
                if not deps:
                    return []
                else:
                    result.extend(deps)

            visited.remove(curr)
            paths[curr] = result
            return result

    # Courses are ordered from 0 to numCourses - 1
    result: list[int] = []
    completed: set[int] = set()
    for course in range(numCourses):
        deps = getOrdering(course)
        deps.reverse()
        if not deps:
            return []
        else:
            for dep in deps:
                if dep not in completed:
                    completed.add(dep)
                    result.append(dep)

    return result

# Testing the solution
assert findOrder(4, [[1,0], [2,0], [3,1], [3,2]]) in ([0,1,2,3], [0,2,1,3])
assert findOrder(5, [[1,0], [2,0], [3,4], [4,3]]) == []

LC Medium: Course Schedule 1 (DFS)

In [8]:
# https://leetcode.com/problems/course-schedule/submissions/1240858033/
def canFinish(numCourses: int, prerequisites: list[list[int]]) -> bool:
    # Convert pre-req to adj list
    adj: dict[int, list[int]] = dict()
    for n1, n2 in prerequisites:
        n1_neighbours = adj.get(n1, [])
        n1_neighbours.append(n2)
        adj[n1] = n1_neighbours

    # DFS Algorithm to check for cycles
    def hasCycle(curr: int, paths: set[int] = set()) -> bool:
        if curr in paths:
            return True
        elif curr in visited:
            return False
        else:
            visited.add(curr)
            paths.add(curr)
            for next_ in adj.get(curr, []):
                if hasCycle(next_, paths):
                    return True
            paths.remove(curr)
            return False

    visited: set[int] = set()
    for curr in range(numCourses):
        if hasCycle(curr):
            return False
    else:
        return True

# Testing the solution
assert canFinish(2, [[1,0]]) == True
assert canFinish(2, [[1,0],[0,1]]) == False

Shortest path in undirected graph with unit weights

In [9]:
def shortestPathUG(n: int, edges: list[tuple[int, int]], src: int) -> list[int]:
    # Create an adj list
    adj: dict[int, list[int]] = dict()
    for n1, n2 in edges:
        n1_neighbours, n2_neighbours = adj.get(n1, []), adj.get(n2, [])
        n1_neighbours.append(n2)
        n2_neighbours.append(n1)
        adj[n1], adj[n2] = n1_neighbours, n2_neighbours

    shortest: list[int] = [-1 for _ in range(n)]
    heap: list[tuple[int, int]] = [(0, src)]
    while heap:
        curr_dist, curr = heapq.heappop(heap)
        shortest[curr] = curr_dist if shortest[curr] == -1 or shortest[curr] > curr_dist else shortest[curr]
        for next_ in adj.get(curr, []):
            if shortest[next_] == -1 or curr_dist + 1 < shortest[next_]:
                heapq.heappush(heap, (curr_dist + 1, next_))

    return shortest

# Testing the solution
assert shortestPathUG(5, [(0, 1), (1, 4), (2, 3), (2, 4), (3, 4)], 1) == [1, 0, 2, 2, 1]

Shortest path in DAG

In [10]:
def shortestPathInDAG(N: int, M: int, edges: list[list[int]]) -> list[int]:
    # Create an adj list
    adj: dict[int, list[tuple[int, int]]] = dict()
    for n1, n2, w in edges:
        n1_neighbours = adj.get(n1, [])
        n1_neighbours.append((n2, w))
        adj[n1] = n1_neighbours

    # Dijkstra's algo!
    shortest: list[int] = [-1 for _ in range(N)]
    heap: list[tuple[int, int]] = [(0, 0)]
    while heap:
        curr_dist, curr = heapq.heappop(heap)
        if shortest[curr] == -1 or curr_dist < shortest[curr]:
            shortest[curr] = curr_dist
            for next_, next_dist in adj.get(curr, []):
                heapq.heappush(heap, (next_dist + curr_dist, next_))

    return shortest

# Testing the solution
assert shortestPathInDAG(3, 3, [[0,1,2], [1,2,3], [0,2,6]]) == [0, 2, 5]
assert shortestPathInDAG(3, 3, [[2,0,4],[0,1,3],[2,1,2]]) == [0, 3, -1]

LC Hard: Word Ladder

In [11]:
# https://leetcode.com/problems/word-ladder/submissions/1241826378
def ladderLengthBrute(beginWord: str, endWord: str, wordList: list[str]) -> int:
    def pathExists(w1: str, w2: str) -> bool:
        N, i = len(w1), 0
        dist = 0
        while i < N:
            if w1[i] != w2[i]:
                dist += 1
            if dist > 1:
                break
            i += 1

        return dist == 1

    def BFS(root: str):
        queue: collections.deque = collections.deque([root])
        distances[root] = 0
        while queue:
            curr = queue.popleft()
            for next_ in adj.get(curr, []):
                if next_ not in distances or distances[next_] > distances[curr] + 1:
                    distances[next_] = distances[curr] + 1
                    queue.append(next_)
                    if next_ == endWord:
                        return

    # Create an adjacency list
    adj: dict[str, list[str]] = dict()
    for w1, w2 in itertools.combinations([beginWord] + wordList, r=2):
        if pathExists(w1, w2):
            w1_neighbours, w2_neighbours = adj.get(w1, []), adj.get(w2, [])
            w1_neighbours.append(w2)
            w2_neighbours.append(w1)
            adj[w1], adj[w2] = w1_neighbours, w2_neighbours

    distances: dict[str, int] = dict()
    BFS(beginWord)
    return distances[endWord] + 1 if endWord in distances else 0

# Testing the solution
assert ladderLengthBrute("hit", "cog", ["hot","dot","dog","lot","log","cog"]) == 5
assert ladderLengthBrute("hit", "cog", ["hot","dot","dog","lot","log"]) == 0

LC Hard: Word Ladder - 2: https://leetcode.com/problems/word-ladder-ii/

In [12]:
# TLE :(
def findLadders(beginWord: str, endWord: str, wordList: list[str]) -> list[list[str]]:
    # BFS traversal to get the list of all shortest prev nodes
    N = len(beginWord)
    wordMap: dict[str, tuple[float, set[str]]] = {word: (math.inf, set()) for word in wordList}
    wordMap[beginWord] = (0, set())
    queue: collections.deque[tuple[str, int, str|None]] = collections.deque([(beginWord, 0, None)])
    while queue:
        curr, dist, prev = queue.popleft()
        for i in range(N):
            for j in range(ord('a'), ord('z') + 1):
                next_ = curr[:i] + chr(j) + curr[i + 1:]
                if next_ in wordMap:
                    next_dist, prevs = wordMap[next_]
                    if dist + 1 < next_dist:
                        prevs = set([curr])
                        next_dist = dist + 1
                        queue.append((next_, dist + 1, curr))
                    elif dist + 1 == next_dist:
                        prevs.add(curr)
                        queue.append((next_, dist + 1, curr))

                    wordMap[next_] = next_dist, prevs

    # Reverse the edges
    adj: dict[str, set[str]] = dict()
    for k, v in wordMap.items():
        n1, n1_neighbours = k, v[1]
        for n2 in n1_neighbours:
            n2_neighbours = adj.get(n2, set())
            n2_neighbours.add(n1)
            adj[n2] = n2_neighbours

    # Do any traversal to get the final result
    results: list[list[str]] = []
    def DFS(root: str, paths: list[str]):
        paths.append(root)
        if root == endWord:
            results.append(list(paths))
        else:
            for next_ in adj.get(root, set()):
                DFS(next_, paths)
        paths.pop()

    DFS(beginWord, [])
    return sorted(results)

# Testing the solution
assert findLadders("hit", "cog", ["hot","dot","dog","lot","log","cog"]) == [['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']]
assert findLadders("hit", "cog", ["hot","dot","dog","lot","log"]) == []
assert findLadders("a", "c", ["a", "b", "c"]) == [["a", "c"]]
assert findLadders("red", "tax", ["ted","tex","red","tax","tad","den","rex","pee"]) == [['red', 'rex', 'tex', 'tax'],['red', 'ted', 'tad', 'tax'],['red', 'ted', 'tex', 'tax']]

LC Biweekly contest: 129 (27th Apr 2024)

In [13]:
# Q1 - Easy
def canMakeSquare(grid: list[list[str]]) -> bool:
    for i, j in [(0, 0), (0, 1), (1, 0), (1, 1)]:
        count = 0
        for x, y in [(0, 0), (0, 1), (1, 0), (1, 1)]:
            i_, j_ = i + x, j + y
            if grid[i_][j_] == "W":
                count += 1
        if count != 2:
            return True
    else:
        return False

# Testing the solution
assert canMakeSquare([["B","W","B"],["B","W","W"],["B","W","W"]]) == True
assert canMakeSquare([["B","W","B"],["W","B","W"],["B","W","B"]]) == False
assert canMakeSquare([["B","B","B"],["B","B","B"],["B","B","B"]]) == True

In [14]:
# Q2 - Medium
def numberOfRightTriangles(grid: list[list[int]]) -> int:
    # Find the dimensions
    M, N = len(grid), len(grid[0])

    # Precompute the number of 1's row & col wise
    rows = [0 for i in range(M)]
    cols = [0 for i in range(N)]
    for i in range(M):
        for j in range(N):
            if grid[i][j] == 1:
                rows[i] += 1
                cols[j] += 1

    # Traverse through the grid again. Each time we encounter a 1, count 1's in that row and excluding itself and find the product
    count = 0
    for i in range(M):
        for j in range(N):
            if grid[i][j] == 1:
                count += (rows[i] - 1) * (cols[j] - 1)

    return count

# Testing the solution
assert numberOfRightTriangles([[1,0,1],[1,0,0],[1,0,0]]) == 2
assert numberOfRightTriangles([[1,0,0,0],[0,1,0,1],[1,0,0,0]]) == 0
assert numberOfRightTriangles([[0,1,0],[0,1,1],[0,1,0]]) == 2

LC Weekly Contest - 395 (28th Apr 2024)

In [15]:
# Q3: Medium
def minEnd(n: int, x: int) -> int:

    # convert to bin
    x_str = bin(x)[2:]
    n_str = bin(n - 1)[2:]

    # compute length of both bins
    xn, nn = len(x_str), len(n_str)

    # store on stack, use two pointers
    stack: list[str] = []
    i, j = xn - 1, nn - 1
    while i >= 0 or j >= 0:
        if j < 0 or (i >= 0 and x_str[i] == '1'):
            stack.append(x_str[i])
            i -= 1
        else:
            stack.append(n_str[j])
            i, j = i - 1, j - 1

    stack.reverse()
    return int(''.join(stack), base=2)

# Testing the solution
assert minEnd(9, 9) == 41
assert minEnd(3, 4) == 6

Practice Bellman Ford, Floyd Warshall

In [16]:
def bellmanFord(N: int, M: int, src: int, edges: list[list[int]]) -> list[float]:
   # Compute the distances and store to a vector
   distances: list[float] = [math.inf for n1 in range(N + 1)]
   distances[src] = 0

   # Relax all edges for N - 1 iterations
   for i in range(N - 1):
       for n1, n2, w in edges:
           distances[n2] = min(distances[n2], distances[n1] + w)

   # Detect negative cycle
   for n1, n2, w in edges:
       if distances[n1] + w < distances[n2]:
           return [-1]

   # Return the distances array
   return distances

In [17]:
def floydWarshall(N: int, M: int, src: int, dest: int, edges: list[list[int]]):
    # Create an adjacency matrix
    distances: list[list[float]] = [[math.inf if i != j else 0 for j in range(N)] for i in range(N)]
    for n1, n2, w in edges:
        distances[n1][n2] = w

    # Run the Floyd Warshall algorithm
    for k in range(N):
        for i in range(N):
            for j in range(N):
                distances[i][j] = min(distances[i][j], distances[i][k] + distances[k][j])

    # Check for cycles (matrix[i][i] < 0)
    for i in range(N):
        if distances[i][i] < 0:
            return []

    # Return the cost matrix
    return distances

Prim's Algorithm for MST

In [18]:
def prim(V: int, adj: list[list[list[int]]]):
    # Initial config
    visited: set[int] = set()
    mst_weight = 0
    heap: list[tuple[int, int]] = [(0, 0)]

    # Prim's algo
    while heap:
        curr_weight, curr = heapq.heappop(heap)
        if curr not in visited:
            visited.add(curr)
            mst_weight += curr_weight
            for next_, next_weight in adj[curr]:
                heapq.heappush(heap, (next_weight, next_))

    return mst_weight

In [19]:
# LC Easy: Reverse string prefix 
# Problem link: https://leetcode.com/problems/reverse-prefix-of-word/submissions/1246431154/
def reversePrefix(word: str, prefix_end: str) -> str:
    result: collections.deque[str] = collections.deque()
    prefixComplete = False
    for ch in word:
        if prefixComplete:
            result.append(ch)
        else:
            result.appendleft(ch)
        if ch == prefix_end:
            prefixComplete = True

    if not prefixComplete:
        result.reverse()

    return ''.join(result)

# Testing the solution
assert reversePrefix("abcdefd", "d") == "dcbaefd"
assert reversePrefix("xyxzxe", "z") == "zxyxxe"
assert reversePrefix("abcd", "z") == "abcd"

In [20]:
# LC Easy: count-of-matches-in-tournament/
# Problem Link: https://leetcode.com/problems/count-of-matches-in-tournament
def numberOfMatches(n: int) -> int:
    # Simply return n - 1
    half = n // 2
    if n == 1:
        return half
    else:
        return half + (numberOfMatches(half + 1) if n % 2 else numberOfMatches(half))

for i in range(1, 201):
    assert numberOfMatches(i) == i - 1

Codechef Weekly contest: 1st May 2024

In [21]:
def alien_or() -> None:
    T = int(input())
    for t in range(T):
        N, K = map(int, input().split())
        set_: set[int] = {1 << i for i in range(K)}
        for n in range(N):
            inp = int(input(), 2)
            if inp in set_:
                set_.remove(inp)
        print("YES" if not set_ else "NO")

In [22]:
# LC Medium: https://leetcode.com/problems/accounts-merge/submissions/1246973276/

In [23]:
def accountsMerge(accounts: list[list[str]]) -> list[list[str]]:
    class DS:
        def __init__(self) -> None:
            self.parent: dict[str, str] = {}
            self.size: dict[str, int] = {}

        def findUltimateParent(self, node: str) -> str:
            if node not in self.parent:
                self.parent[node] = node
                self.size[node] = 1

            if self.parent[node] == node:
                return node
            else:
                self.parent[node] = self.findUltimateParent(self.parent[node])
                return self.parent[node]

        def unionBySize(self, u: str, v: str) -> None:
            up, vp = self.findUltimateParent(u), self.findUltimateParent(v)
            ups, vps = self.size[up], self.size[vp]
            if up == vp:
                return
            if ups < vps:
                self.parent[up] = vp
                self.size[vp] += self.size[up]
            else:
                self.parent[vp] = up
                self.size[up] += self.size[vp]

    ds = DS()
    emails: dict[str, str] = dict()
    for account in accounts:
        account = account + [account[-1]]
        name = account[0]
        for i in range(1, len(account) - 1):
            n1, n2 = account[i], account[i + 1]
            emails[n1] = emails[n2] = name
            ds.unionBySize(n1, n2)

    groups: dict[str, list[str]] = dict()
    for email in emails:
        uemail = ds.findUltimateParent(email)
        group = groups.get(uemail, [emails[uemail]])
        group.append(email)
        groups[uemail] = group

    result: list[list[str]] = []
    for group in groups.values():
        result.append([group[0], *sorted(group[1:])])

    return result

# Testing the solution
accountsMerge([
    ["John","johnsmith@mail.com","john_newyork@mail.com"],
    ["John","johnsmith@mail.com","john00@mail.com"],
    ["Mary","mary@mail.com"],
    ["John","johnnybravo@mail.com"]
])

[['John', 'john00@mail.com', 'john_newyork@mail.com', 'johnsmith@mail.com'],
 ['Mary', 'mary@mail.com'],
 ['John', 'johnnybravo@mail.com']]

Codeforces contest: 1968 (2nd May 2024)

In [24]:
# Problem: A
def QA_1968() -> None:
    T = int(input())
    results: list[int] = []
    for _ in range(T):
        x = int(input())
        result: tuple[int, int] = (-1, -1)
        for y in range(1, x):
            curr = math.gcd(y, x) + y
            if curr > result[1]:
                result = y, curr

        results.append(result[0])

    for res in results:
        print(res)

LC Medium: Remove stones
https://leetcode.com/problems/most-stones-removed-with-same-row-or-column/submissions/1248384399/

In [25]:
def removeStones(stones: list[list[int]]) -> int:
    """Max stones that can be removed is (island_size - 1)"""
    class DS:
        def __init__(self) -> None:
            self.size: dict[str, int] = {}
            self.parents: dict[str, str] = {}

        def findUltimateParent(self, node: str) -> str:
            if node not in self.parents:
                self.parents[node] = node
                self.size[node] = 1 if 'r' in node else 0

            if self.parents[node] == node:
                return node
            else:
                self.parents[node] = self.findUltimateParent(self.parents[node])
                return self.parents[node]

        def unionBySize(self, row: str, col: str):
            rp, cp = self.findUltimateParent(row), self.findUltimateParent(col)
            rps, cps = self.size[rp], self.size[cp]
            if rp == cp:
                return
            elif rps < cps:
                self.parents[rp] = cp
                self.size[cp] += self.size[rp]
            else:
                self.parents[cp] = rp
                self.size[rp] += self.size[cp]

    ds = DS()
    for i, j in stones:
        row, col = f'r{i}', f'c{j}'
        ds.unionBySize(row, col)

    # Reset the size, find unique parents
    ds.size.clear()
    parents: set[str] = set()
    for i, _ in stones:
        row = f'r{i}'
        up = ds.findUltimateParent(row)
        ds.size[up] = ds.size.get(up, 0) + 1
        parents.add(up)

    total = 0
    for parent in parents:
        total += ds.size[parent] - 1

    return total

# Testing the solution
assert removeStones([[0,0],[0,2],[1,3],[3,1],[3,2],[4,3]]) == 4
assert removeStones([[0,0],[1,1],[2,2],[2,3],[2,4],[3,2]]) == 3